## Setup


## Import DataSet

In [ ]:
from glob import glob
import os
import shutil
import pandas as pd
from tqdm.notebook import tqdm

Setup kaggle and download kaggle.json the same folder as this script using the following link for setting up kaggle https://www.kaggle.com/docs/api

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download --unzip --force nih-chest-xrays/data/ 

In [ ]:
dataset_folder = '.'
image_files = glob(os.path.join(dataset_folder,'images_0*/images/*.png'))
image_files = pd.Series(image_files).str.rsplit('/',n=1,expand=True)
image_files.columns=['Folder','Image Index']

images = pd.read_csv(os.path.join(dataset_folder,'Data_Entry_2017.csv'))
images = image_files.merge(images.join(images['Finding Labels'].str.get_dummies()),on='Image Index',how='left')

In [ ]:
pos_samples = images[images['Pneumonia']==1]
pos_samples = pos_samples[['Folder','Image Index','Pneumonia']]
neg_samples = images[images['No Finding']==1]
neg_samples = neg_samples[['Folder','Image Index','Pneumonia']]
dataset = pd.concat([pos_samples,neg_samples])
dataset.rename(columns={'Pneumonia':'label'},inplace=True)

In [ ]:
if not os.path.exists(os.path.join(dataset_folder,'training')):
    os.mkdir(os.path.join(dataset_folder,'training'))
    labels = dataset.label.unique()
    for label in labels:
        os.mkdir(os.path.join(dataset_folder,'training',str(label)))

In [ ]:
for rowitem in tqdm(dataset.iterrows(),total=len(dataset)):
    shutil.copy(os.path.join(rowitem[1]['Folder'],rowitem[1]['Image Index']),
                os.path.join(dataset_folder,'training',str(rowitem[1]['label']),rowitem[1]['Image Index']))